# Chapter 9: Recurrent Neural Networks

This chapter introduces recurrent neural networks (RNNs), which are deep learning models that capture the dynamics of sequences via *recurrent* connections. RNNs can be thought of as feedforward neural networks where each layer's parameters are shared across time steps.

🔑 **KEY INSIGHT**: Unlike feedforward networks, RNNs maintain a *hidden state* that acts as memory, allowing them to process sequences of variable length while keeping the number of parameters constant.

---
## 9.1 Working with Sequences

We focus on inputs that consist of an ordered list of feature vectors, where each is indexed by a time step. The key challenge is that the number of inputs varies depending on the sequence length.

In [ ]:
%matplotlib inline
from d2l import torch as d2l
import torch
from torch import nn

### Autoregressive Models

🔑 **KEY INSIGHT**: Autoregressive models predict the next value based on previous observations. The key challenge is that the number of inputs grows with time, so we either (1) use a fixed-length window, or (2) maintain a summary hidden state.

### Training

We create synthetic data following the `sin` function with additive noise.

In [ ]:
class Data(d2l.DataModule):
    def __init__(self, batch_size=16, T=1000, num_train=600, tau=4):
        self.save_hyperparameters()
        self.time = d2l.arange(1, T + 1, dtype=d2l.float32)
        self.x = d2l.sin(0.01 * self.time) + d2l.randn(T) * 0.2

In [ ]:
data = Data()
d2l.plot(data.time, data.x, 'time', 'x', xlim=[1, 1000], figsize=(6, 3))

In [ ]:
@d2l.add_to_class(Data)
def get_dataloader(self, train):
    features = [self.x[i : self.T-self.tau+i] for i in range(self.tau)]
    self.features = d2l.stack(features, 1)
    self.labels = d2l.reshape(self.x[self.tau:], (-1, 1))
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader([self.features, self.labels], train, i)

In [ ]:
model = d2l.LinearRegression(lr=0.01)
trainer = d2l.Trainer(max_epochs=5)
trainer.fit(model, data)

### Prediction

🔑 **KEY INSIGHT**: One-step-ahead prediction works well, but multi-step prediction degrades rapidly because errors accumulate—each prediction feeds into the next, compounding mistakes.

In [ ]:
onestep_preds = d2l.numpy(model(data.features))
d2l.plot(data.time[data.tau:], [data.labels, onestep_preds], 'time', 'x',
         legend=['labels', '1-step preds'], figsize=(6, 3))

In [ ]:
multistep_preds = d2l.zeros(data.T)
multistep_preds[:] = data.x
for i in range(data.num_train + data.tau, data.T):
    multistep_preds[i] = model(
        d2l.reshape(multistep_preds[i-data.tau : i], (1, -1)))
multistep_preds = d2l.numpy(multistep_preds)

In [ ]:
d2l.plot([data.time[data.tau:], data.time[data.num_train+data.tau:]],
         [onestep_preds, multistep_preds[data.num_train+data.tau:]], 'time',
         'x', legend=['1-step preds', 'multistep preds'], figsize=(6, 3))

In [ ]:
def k_step_pred(k):
    features = []
    for i in range(data.tau):
        features.append(data.x[i : i+data.T-data.tau-k+1])
    # The (i+tau)-th element stores the (i+1)-step-ahead predictions
    for i in range(k):
        preds = model(d2l.stack(features[i : i+data.tau], 1))
        features.append(d2l.reshape(preds, -1))
    return features[data.tau:]

In [ ]:
steps = (1, 4, 16, 64)
preds = k_step_pred(steps[-1])
d2l.plot(data.time[data.tau+steps[-1]-1:],
         [d2l.numpy(preds[k-1]) for k in steps], 'time', 'x',
         legend=[f'{k}-step preds' for k in steps], figsize=(6, 3))

---
## 9.2 Converting Raw Text into Sequence Data

Text preprocessing pipeline: (1) Load text as strings, (2) Split into tokens, (3) Build vocabulary mapping tokens to indices, (4) Convert text to numerical indices.

In [ ]:
import collections
import re
from d2l import torch as d2l
import torch
import random

### Reading the Dataset

In [ ]:
class TimeMachine(d2l.DataModule):  #@save
    """The Time Machine dataset."""
    def _download(self):
        fname = d2l.download(d2l.DATA_URL + 'timemachine.txt', self.root,
                             '090b5e7e70c295757f55df93cb0a180b9691891a')
        with open(fname) as f:
            return f.read()

data = TimeMachine()
raw_text = data._download()
raw_text[:60]

In [ ]:
@d2l.add_to_class(TimeMachine)  #@save
def _preprocess(self, text):
    return re.sub('[^A-Za-z]+', ' ', text).lower()

text = data._preprocess(raw_text)
text[:60]

### Tokenization

In [ ]:
@d2l.add_to_class(TimeMachine)  #@save
def _tokenize(self, text):
    return list(text)

tokens = data._tokenize(text)
','.join(tokens[:30])

### Vocabulary

🔑 **KEY INSIGHT**: A vocabulary maps each unique token to a numerical index. Rare tokens are often replaced with a special `<unk>` token to keep the vocabulary manageable.

In [ ]:
class Vocab:  #@save
    """Vocabulary for text."""
    def __init__(self, tokens=[], min_freq=0, reserved_tokens=[]):
        # Flatten a 2D list if needed
        if tokens and isinstance(tokens[0], list):
            tokens = [token for line in tokens for token in line]
        # Count token frequencies
        counter = collections.Counter(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # The list of unique tokens
        self.idx_to_token = list(sorted(set(['<unk>'] + reserved_tokens + [
            token for token, freq in self.token_freqs if freq >= min_freq])))
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if hasattr(indices, '__len__') and len(indices) > 1:
            return [self.idx_to_token[int(index)] for index in indices]
        return self.idx_to_token[indices]

    @property
    def unk(self):  # Index for the unknown token
        return self.token_to_idx['<unk>']

In [ ]:
vocab = Vocab(tokens)
indices = vocab[tokens[:10]]
print('indices:', indices)
print('words:', vocab.to_tokens(indices))

### Putting It All Together

In [ ]:
@d2l.add_to_class(TimeMachine)  #@save
def build(self, raw_text, vocab=None):
    tokens = self._tokenize(self._preprocess(raw_text))
    if vocab is None: vocab = Vocab(tokens)
    corpus = [vocab[token] for token in tokens]
    return corpus, vocab

corpus, vocab = data.build(raw_text)
len(corpus), len(vocab)

### Exploratory Language Statistics

🔑 **KEY INSIGHT**: Word frequencies follow Zipf's law—the frequency of a word is inversely proportional to its rank. This holds for unigrams, bigrams, and trigrams.

In [ ]:
words = text.split()
vocab = Vocab(words)
vocab.token_freqs[:10]

In [ ]:
freqs = [freq for token, freq in vocab.token_freqs]
d2l.plot(freqs, xlabel='token: x', ylabel='frequency: n(x)',
         xscale='log', yscale='log')

In [ ]:
bigram_tokens = ['--'.join(pair) for pair in zip(words[:-1], words[1:])]
bigram_vocab = Vocab(bigram_tokens)
bigram_vocab.token_freqs[:10]

In [ ]:
trigram_tokens = ['--'.join(triple) for triple in zip(
    words[:-2], words[1:-1], words[2:])]
trigram_vocab = Vocab(trigram_tokens)
trigram_vocab.token_freqs[:10]

In [ ]:
bigram_freqs = [freq for token, freq in bigram_vocab.token_freqs]
trigram_freqs = [freq for token, freq in trigram_vocab.token_freqs]
d2l.plot([freqs, bigram_freqs, trigram_freqs], xlabel='token: x',
         ylabel='frequency: n(x)', xscale='log', yscale='log',
         legend=['unigram', 'bigram', 'trigram'])

---
## 9.3 Language Models

The goal of language models is to estimate the joint probability of a sequence: $P(x_1, x_2, \ldots, x_T)$.

🔑 **KEY INSIGHT**: We decompose the joint probability using the chain rule: $P(x_1, \ldots, x_T) = \prod_{t=1}^T P(x_t | x_{t-1}, \ldots, x_1)$. This allows autoregressive modeling.

In [ ]:
from d2l import torch as d2l
import torch

### Partitioning Sequences

🔑 **KEY INSIGHT**: For training, we partition sequences into fixed-length subsequences. The target is simply the input shifted by one token—we predict the next token given previous tokens.

In [ ]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def __init__(self, batch_size, num_steps, num_train=10000, num_val=5000):
    super(d2l.TimeMachine, self).__init__()
    self.save_hyperparameters()
    corpus, self.vocab = self.build(self._download())
    array = d2l.tensor([corpus[i:i+num_steps+1]
                        for i in range(len(corpus)-num_steps)])
    self.X, self.Y = array[:,:-1], array[:,1:]

In [ ]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(
        self.num_train, self.num_train + self.num_val)
    return self.get_tensorloader([self.X, self.Y], train, idx)

In [ ]:
data = d2l.TimeMachine(batch_size=2, num_steps=10)
for X, Y in data.train_dataloader():
    print('X:', X, '\nY:', Y)
    break

---
## 9.4 Recurrent Neural Networks

🔑 **KEY INSIGHT**: RNNs use a *hidden state* $h_t$ to compress all sequence information up to time $t$. The hidden state is updated as: $h_t = f(x_t, h_{t-1})$. This allows handling arbitrarily long sequences with fixed parameters.

In [ ]:
from d2l import torch as d2l
import torch

### RNNs with Hidden States

The hidden state equation: $\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h)$

🔑 **KEY INSIGHT**: The computation $\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$ is equivalent to concatenating inputs and hidden state, then multiplying by a single weight matrix.

In [ ]:
X, W_xh = d2l.randn(3, 1), d2l.randn(1, 4)
H, W_hh = d2l.randn(3, 4), d2l.randn(4, 4)
d2l.matmul(X, W_xh) + d2l.matmul(H, W_hh)

In [ ]:
d2l.matmul(d2l.concat((X, H), 1), d2l.concat((W_xh, W_hh), 0))

---
## 9.5 RNN Implementation from Scratch

We implement an RNN from scratch and train it as a character-level language model.

In [ ]:
%matplotlib inline
from d2l import torch as d2l
import math
import torch
from torch import nn
from torch.nn import functional as F

### RNN Model

In [ ]:
class RNNScratch(d2l.Module):  #@save
    """The RNN model implemented from scratch."""
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.W_xh = nn.Parameter(
            d2l.randn(num_inputs, num_hiddens) * sigma)
        self.W_hh = nn.Parameter(
            d2l.randn(num_hiddens, num_hiddens) * sigma)
        self.b_h = nn.Parameter(d2l.zeros(num_hiddens))

In [ ]:
@d2l.add_to_class(RNNScratch)  #@save
def forward(self, inputs, state=None):
    if state is None:
        # Initial state with shape: (batch_size, num_hiddens)
        state = d2l.zeros((inputs.shape[1], self.num_hiddens),
                          device=inputs.device)
    else:
        state, = state
    outputs = []
    for X in inputs:  # Shape of inputs: (num_steps, batch_size, num_inputs)
        state = d2l.tanh(d2l.matmul(X, self.W_xh) +
                         d2l.matmul(state, self.W_hh) + self.b_h)
        outputs.append(state)
    return outputs, state

In [ ]:
batch_size, num_inputs, num_hiddens, num_steps = 2, 16, 32, 100
rnn = RNNScratch(num_inputs, num_hiddens)
X = d2l.ones((num_steps, batch_size, num_inputs))
outputs, state = rnn(X)

In [ ]:
def check_len(a, n):  #@save
    """Check the length of a list."""
    assert len(a) == n, f'list\'s length {len(a)} != expected length {n}'

def check_shape(a, shape):  #@save
    """Check the shape of a tensor."""
    assert a.shape == shape, \
            f'tensor\'s shape {a.shape} != expected shape {shape}'

check_len(outputs, num_steps)
check_shape(outputs[0], (batch_size, num_hiddens))
check_shape(state, (batch_size, num_hiddens))

### RNN-Based Language Model

In [ ]:
class RNNLMScratch(d2l.Classifier):  #@save
    """The RNN-based language model implemented from scratch."""
    def __init__(self, rnn, vocab_size, lr=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.init_params()

    def init_params(self):
        self.W_hq = nn.Parameter(
            d2l.randn(
                self.rnn.num_hiddens, self.vocab_size) * self.rnn.sigma)
        self.b_q = nn.Parameter(d2l.zeros(self.vocab_size))

    def training_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('ppl', d2l.exp(l), train=True)
        return l

    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('ppl', d2l.exp(l), train=False)

### One-Hot Encoding

🔑 **KEY INSIGHT**: We use one-hot encoding to represent tokens because treating token indices as numerical values would incorrectly imply ordering relationships between unrelated tokens.

In [ ]:
F.one_hot(torch.tensor([0, 2]), 5)

In [ ]:
@d2l.add_to_class(RNNLMScratch)  #@save
def one_hot(self, X):
    # Output shape: (num_steps, batch_size, vocab_size)
    return F.one_hot(X.T, self.vocab_size).type(torch.float32)

### Transforming RNN Outputs

In [ ]:
@d2l.add_to_class(RNNLMScratch)  #@save
def output_layer(self, rnn_outputs):
    outputs = [d2l.matmul(H, self.W_hq) + self.b_q for H in rnn_outputs]
    return d2l.stack(outputs, 1)

@d2l.add_to_class(RNNLMScratch)  #@save
def forward(self, X, state=None):
    embs = self.one_hot(X)
    rnn_outputs, _ = self.rnn(embs, state)
    return self.output_layer(rnn_outputs)

In [ ]:
model = RNNLMScratch(rnn, num_inputs)
outputs = model(d2l.ones((batch_size, num_steps), dtype=d2l.int64))
check_shape(outputs, (batch_size, num_steps, num_inputs))

### Gradient Clipping

🔑 **KEY INSIGHT**: RNNs suffer from exploding gradients due to repeated matrix multiplications through time. Gradient clipping projects gradients onto a ball of radius θ: $\mathbf{g} \leftarrow \min(1, \frac{\theta}{\|\mathbf{g}\|}) \mathbf{g}$

In [ ]:
@d2l.add_to_class(d2l.Trainer)  #@save
def clip_gradients(self, grad_clip_val, model):
    params = [p for p in model.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > grad_clip_val:
        for param in params:
            param.grad[:] *= grad_clip_val / norm

### Training

In [ ]:
data = d2l.TimeMachine(batch_size=1024, num_steps=32)
rnn = RNNScratch(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLMScratch(rnn, vocab_size=len(data.vocab), lr=1)
trainer = d2l.Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

### Decoding

🔑 **KEY INSIGHT**: During generation, we use a "warm-up" period where we feed the prefix to build up the hidden state, then generate one token at a time by feeding each prediction back as the next input.

In [ ]:
@d2l.add_to_class(RNNLMScratch)  #@save
def predict(self, prefix, num_preds, vocab, device=None):
    state, outputs = None, [vocab[prefix[0]]]
    for i in range(len(prefix) + num_preds - 1):
        X = d2l.tensor([[outputs[-1]]], device=device)
        embs = self.one_hot(X)
        rnn_outputs, state = self.rnn(embs, state)
        if i < len(prefix) - 1:  # Warm-up period
            outputs.append(vocab[prefix[i + 1]])
        else:  # Predict num_preds steps
            Y = self.output_layer(rnn_outputs)
            outputs.append(int(d2l.reshape(d2l.argmax(Y, axis=2), 1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [ ]:
model.predict('it has', 20, data.vocab, d2l.try_gpu())

---
## 9.6 Concise Implementation of RNNs

Using high-level APIs for faster, optimized implementations.

In [ ]:
from d2l import torch as d2l
import torch
from torch import nn
from torch.nn import functional as F

### Defining the Model

In [ ]:
class RNN(d2l.Module):  #@save
    """The RNN model implemented with high-level APIs."""
    def __init__(self, num_inputs, num_hiddens):
        super().__init__()
        self.save_hyperparameters()
        self.rnn = nn.RNN(num_inputs, num_hiddens)

    def forward(self, inputs, H=None):
        return self.rnn(inputs, H)

In [ ]:
class RNNLM(d2l.RNNLMScratch):  #@save
    """The RNN-based language model implemented with high-level APIs."""
    def init_params(self):
        self.linear = nn.LazyLinear(self.vocab_size)

    def output_layer(self, hiddens):
        return d2l.swapaxes(self.linear(hiddens), 0, 1)

### Training and Predicting

In [ ]:
data = d2l.TimeMachine(batch_size=1024, num_steps=32)
rnn = RNN(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLM(rnn, vocab_size=len(data.vocab), lr=1)
model.predict('it has', 20, data.vocab)

In [ ]:
trainer = d2l.Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [ ]:
model.predict('it has', 20, data.vocab, d2l.try_gpu())

---
## 9.7 Backpropagation Through Time

Backpropagation through time (BPTT) is how we compute gradients in RNNs by unrolling the network across time steps.

🔑 **KEY INSIGHT**: The gradient $\partial h_t / \partial w_h$ involves a product of matrices across all time steps. If eigenvalues are < 1, gradients vanish; if > 1, gradients explode. This is why we need gradient clipping and why LSTMs/GRUs were invented.

### Strategies for Computing Gradients:

1. **Full Computation**: Compute the entire sum—computationally expensive and numerically unstable
2. **Truncated BPTT**: Only backpropagate through τ time steps—practical and commonly used
3. **Randomized Truncation**: Randomly truncate with correct expectation—theoretically nice but not much better in practice

🔑 **KEY INSIGHT**: Truncated BPTT acts as a regularizer, biasing the model toward shorter-range dependencies. This is often desirable in practice.

---
## Summary

Key takeaways from Chapter 9:

1. **Sequences require special handling**: The variable-length nature of sequences requires either fixed windows or hidden states to maintain context.

2. **RNNs use recurrent connections**: Hidden state $h_t = f(x_t, h_{t-1})$ captures sequence history with constant parameters.

3. **Language modeling is autoregressive**: Predict next token given previous tokens using chain rule decomposition.

4. **Gradient clipping is essential**: Prevents exploding gradients that destabilize training.

5. **Perplexity measures model quality**: $\exp(-\frac{1}{n}\sum_t \log P(x_t|x_{<t}))$ - lower is better.

6. **Truncated BPTT is practical**: Full backpropagation is unstable; truncating provides a good tradeoff.